In [1]:
#1

import pandas as pd
import psycopg2 as psy
import warnings

warnings.filterwarnings("ignore")

conn = psy.connect(database='corgirace', \
                   user='dannysevere', \
                   password='Severe21!', \
                   host='dbsevere.ct4e9f3blg2v.us-west-1.rds.amazonaws.com', \
                   port='5432')

cursor = conn.cursor()

df = pd.read_sql('SELECT * FROM CORGI', conn, index_col='corgid')
print(df)

                         corgname  breed gender  weight  age
corgid                                                      
1                         Brookie    Pem      F      21  3.5
2                   Golden Graham    Pem      M      30  2.0
3                          Kraken    Pem     SF      22  3.5
4                     Snarla June    Pem     SF      29  3.5
5                           Isaac  Cardi     NM      33  4.5
6                         Stanley    Pem      M      33  3.5
7                         Optimus    Pem      M      25  4.0
8                        Gu∂ri∂ur    Pem      F      20  5.0
9       Baroness von Fluffyshanks    Pem     SF      22  4.0
10                     Prometheus    Pem      M      27  5.0
11                 Foxy Stumptail    Pem     NM      29  5.5
12                      Drederick    Pem      M      29  4.5
13              Lady Wigglebottom    Pem      F      28  4.0
14                       Jedediah    Pem     NM      30  4.0
15                   Mis

In [2]:
#2

query = '''WITH cte_snar AS (SELECT *
            FROM CORGI as c JOIN OUTCOME as o ON c.corgid = o.corgid
            where corgname = 'Snarla June')
            
            SELECT racename, racestate, EXTRACT(MONTH FROM racedate) AS race_month, racetemp, racetime
            FROM RACE as r JOIN cte_snar as s ON r.raceid = s.raceid
            '''

df = pd.read_sql(query, conn)
print(df)

df_mat = df[['racetime', 'race_month', 'racetemp']]

print('\n', df_mat.corr())
print('\nTheres a moderate correlation between racetime and both race_month and race_temp. For higher temperatures, racetime increases and for later months, racetime decreases.\n\nThis indicates a relationship between better performance and later, cooler months.')


                   racename racestate  race_month  racetemp  racetime
0            Del Mar Trophy        CA         6.0        80     64.95
1            Memorial Derby        WA         6.0        77     56.27
2               Dixie Derby        UT         6.0        95     69.77
3       Billings Grand Prix        MT         8.0        95     65.80
4                 Idaho Cup        ID         8.0        93     62.79
5           Bachelor Stakes        OR         8.0        87     59.25
6  Golden Gate Invitational        CA         9.0        81     40.10

             racetime  race_month  racetemp
racetime    1.000000   -0.580836  0.589349
race_month -0.580836    1.000000  0.196781
racetemp    0.589349    0.196781  1.000000

Theres a moderate correlation between racetime and both race_month and race_temp. For higher temperatures, racetime increases and for later months, racetime decreases.

This indicates a relationship between better performance and later, cooler months.


In [3]:
#3

query = ''' 
            SELECT corgname, COUNT(*) AS race_cnt, ROUND(AVG(racetime), 1) AS avg_racetime
            FROM CORGI AS c JOIN OUTCOME AS o ON c.corgid = o.corgid
            GROUP BY corgname
            HAVING AVG(racetime) < 57
            ORDER BY AVG(racetime)
            ;'''

df = pd.read_sql(query, conn)

print(df)

                    corgname  race_cnt  avg_racetime
0                      Bucco         4          48.6
1                   Fruitcup         5          52.7
2              Golden Graham         4          54.2
3                       Crow         6          54.8
4  Baroness von Fluffyshanks         4          55.2
5                Gareth Bale         4          55.3
6                      Sempo         6          55.9
7              Winston Smith         4          56.0
8        Mary Queen of Corgs         4          56.9


In [4]:
#4
query = ''' 
           SELECT racename, racedate, COUNT(*) AS comp_cnt, MIN(racetime) AS best, MAX(racetime) AS worst
           FROM RACE as r JOIN OUTCOME as o ON r.raceid = o.raceid 
           GROUP BY racename, racedate
           ORDER BY racedate
            ;'''

df = pd.read_sql(query, conn)

print(df)

                    racename    racedate  comp_cnt   best  worst
0             Del Mar Trophy  2018-06-03        12  41.26  71.46
1             Memorial Derby  2018-06-17        10  44.89  71.54
2                Dixie Derby  2018-06-24        12  40.74  72.79
3               Missions Cup  2018-06-30        13  40.73  71.80
4            Borderside Dash  2018-07-21        11  42.75  74.28
5               Teton Runoff  2018-07-29        14  48.33  73.49
6              Emerald Chase  2018-08-05        12  42.99  72.26
7        Billings Grand Prix  2018-08-12        11  42.59  74.93
8                  Idaho Cup  2018-08-19        13  44.00  73.36
9            Bachelor Stakes  2018-08-26        12  40.07  73.30
10  Golden Gate Invitational  2018-09-23        14  40.10  74.54


In [5]:
#5

query = '''WITH ct AS (
           SELECT racename, racedate, MIN(racetime) AS best, MAX(racetime) AS worst
           FROM RACE as r JOIN OUTCOME as o ON r.raceid = o.raceid 
           GROUP BY racename, racedate
           ORDER BY racedate
           ),
           
           ct2 AS (
           SELECT racename, racedate, worst - best AS time_difference
           FROM ct
           ORDER BY time_difference DESC
           LIMIT 3
           )
           
           SELECT racename, racedate, time_difference
           FROM ct2
            ;'''

df = pd.read_sql(query, conn)

print(df)

                   racename    racedate  time_difference
0  Golden Gate Invitational  2018-09-23            34.44
1           Bachelor Stakes  2018-08-26            33.23
2       Billings Grand Prix  2018-08-12            32.34


In [6]:
#6

query = '''WITH ct AS (
           SELECT racename, racetime
           FROM CORGI c JOIN OUTCOME o ON c.corgid = o.corgid JOIN RACE r ON r.raceid = o.raceid
           WHERE corgname = 'Stanley')
           
           SELECT r.racename, racedate, racetemp, ct.racetime AS racetime_for_stanley
           FROM RACE r LEFT JOIN ct ON r.racename = ct.racename
           ORDER BY racedate
            ;'''

df = pd.read_sql(query, conn)

print(df)

                    racename    racedate  racetemp  racetime_for_stanley
0             Del Mar Trophy  2018-06-03        80                 66.97
1             Memorial Derby  2018-06-17        77                   NaN
2                Dixie Derby  2018-06-24        95                 44.68
3               Missions Cup  2018-06-30        73                 49.93
4            Borderside Dash  2018-07-21        74                 74.28
5               Teton Runoff  2018-07-29        77                   NaN
6              Emerald Chase  2018-08-05        89                 62.83
7        Billings Grand Prix  2018-08-12        95                   NaN
8                  Idaho Cup  2018-08-19        93                   NaN
9            Bachelor Stakes  2018-08-26        87                   NaN
10  Golden Gate Invitational  2018-09-23        81                 58.02


In [7]:
#7

query = '''WITH ct AS (
            SELECT o.raceid, racename, AVG(racetime) - 5 AS sub5, COUNT(*) as crg_cnt
            FROM OUTCOME O JOIN RACE R ON O.RACEID = R.RACEID
            GROUP BY RACENAME, o.raceid
            ),
            
            ccnt AS (
            SELECT o.raceid, racename, COUNT(*) as fstcrg_cnt
            FROM OUTCOME O JOIN ct ON O.raceid = ct.raceid
            WHERE racetime <= sub5
            GROUP BY o.raceid, racename
            )
            
            SELECT c.racename, AVG(racetime) as avg_racetime, sub5, fstcrg_cnt, crg_cnt, (CAST(fstcrg_cnt AS DECIMAL) / crg_cnt) * 100 AS fstcrg_percent
            FROM ct JOIN OUTCOME o ON o.raceid = ct.raceid JOIN ccnt AS c ON c.raceid = ct.raceid
            GROUP BY c.racename, sub5, fstcrg_cnt, crg_cnt
            
            
            
            
            
            
            
            

            ;'''

df = pd.read_sql(query, conn)

print(df)

                    racename  avg_racetime       sub5  fstcrg_cnt  crg_cnt  \
0            Borderside Dash     58.186364  53.186364           3       11   
1               Missions Cup     57.786923  52.786923           5       13   
2   Golden Gate Invitational     58.482143  53.482143           5       14   
3               Teton Runoff     61.490714  56.490714           5       14   
4              Emerald Chase     58.328333  53.328333           4       12   
5                Dixie Derby     59.446667  54.446667           4       12   
6                  Idaho Cup     57.919231  52.919231           4       13   
7            Bachelor Stakes     59.707500  54.707500           3       12   
8             Memorial Derby     60.780000  55.780000           3       10   
9        Billings Grand Prix     58.852727  53.852727           4       11   
10            Del Mar Trophy     57.383333  52.383333           4       12   

    fstcrg_percent  
0        27.272727  
1        38.461538  


In [8]:
#8

query = ''' SELECT racename, racedate, COUNT(CASE WHEN gender = 'M' or gender = 'NM' THEN 1 END) AS males, COUNT(CASE WHEN gender = 'F' or gender = 'SF' THEN 1 END) AS females
            FROM CORGI c JOIN OUTCOME o ON c.corgid = o.corgid JOIN RACE r ON o.raceid = r.raceid
            GROUP BY racename, racedate
            ORDER BY racedate
            
            
            ;'''

df = pd.read_sql(query, conn)

print(df)

                    racename    racedate  males  females
0             Del Mar Trophy  2018-06-03      8        4
1             Memorial Derby  2018-06-17      4        6
2                Dixie Derby  2018-06-24      5        7
3               Missions Cup  2018-06-30      8        5
4            Borderside Dash  2018-07-21      8        3
5               Teton Runoff  2018-07-29      8        6
6              Emerald Chase  2018-08-05      9        3
7        Billings Grand Prix  2018-08-12      4        7
8                  Idaho Cup  2018-08-19      8        5
9            Bachelor Stakes  2018-08-26      6        6
10  Golden Gate Invitational  2018-09-23      7        7


In [9]:
#9

query = ''' WITH ct AS (
            SELECT racename, racedate, o.raceid, MIN(racetime) winning_time
            FROM OUTCOME o JOIN RACE r ON o.raceid = r.raceid
            GROUP BY racename, racedate, o.raceid
            )
            
            SELECT  racename, racedate, corgname, winning_time
            FROM CORGI c JOIN OUTCOME o ON c.corgid = o.corgid JOIN ct ON o.raceid = ct.raceid AND racetime = winning_time
            ORDER BY racedate
            ;'''

df = pd.read_sql(query, conn)

print(df)


                    racename    racedate                   corgname  \
0             Del Mar Trophy  2018-06-03                       Crow   
1             Memorial Derby  2018-06-17                      Bucco   
2                Dixie Derby  2018-06-24  Baroness von Fluffyshanks   
3               Missions Cup  2018-06-30                      Bucco   
4            Borderside Dash  2018-07-21                Gareth Bale   
5               Teton Runoff  2018-07-29                    Oatmeal   
6              Emerald Chase  2018-08-05                   Fruitcup   
7        Billings Grand Prix  2018-08-12                      Sempo   
8                  Idaho Cup  2018-08-19  Baroness von Fluffyshanks   
9            Bachelor Stakes  2018-08-26                   Jedediah   
10  Golden Gate Invitational  2018-09-23                Snarla June   

    winning_time  
0          41.26  
1          44.89  
2          40.74  
3          40.73  
4          42.75  
5          48.33  
6          42.

In [11]:
#10

x = 0 

while x == 0:
    try:
        usr = int(input('enter a corgid: '))
        
        if usr >= 1 and usr <= 25:

            df = pd.read_sql('SELECT corgname FROM CORGI WHERE corgid = %d'% (usr), conn)
            
            print('\n',df.loc[0])
            
            df2 = pd.read_sql('''
            WITH ct AS(
            SELECT o.raceid, racename, racedate, COUNT(*) as crg_cnt
            FROM OUTCOME O JOIN RACE R ON O.RACEID = R.RACEID
            GROUP BY RACENAME, o.raceid, racedate),
            
            ct2 AS (
            SELECT raceid, racetime AS usr_corgtime
            FROM OUTCOME
            WHERE corgid = %d)
            
            SELECT racename, racedate, crg_cnt, usr_corgtime
            FROM ct LEFT JOIN ct2 ON ct.raceid = ct2.raceid
            ORDER BY racedate
                                  
                              
                              
                              '''% (usr), conn)
            print(df2)
            x = 1
           
             
        else:
            print('Invalid corgid. Try again')
            
    except:
        print('Invalid input')


enter a corgid: 78
Invalid corgid. Try again
enter a corgid: dfjkl
Invalid input
enter a corgid: 22

 corgname    Fruitcup
Name: 0, dtype: object
                    racename    racedate  crg_cnt  usr_corgtime
0             Del Mar Trophy  2018-06-03       12           NaN
1             Memorial Derby  2018-06-17       10         47.97
2                Dixie Derby  2018-06-24       12           NaN
3               Missions Cup  2018-06-30       13         50.34
4            Borderside Dash  2018-07-21       11           NaN
5               Teton Runoff  2018-07-29       14         67.37
6              Emerald Chase  2018-08-05       12         42.99
7        Billings Grand Prix  2018-08-12       11         54.61
8                  Idaho Cup  2018-08-19       13           NaN
9            Bachelor Stakes  2018-08-26       12           NaN
10  Golden Gate Invitational  2018-09-23       14           NaN


##### 